In [9]:
import dvc.api
import flatten_dict

In [10]:
branch_a = "dvc-base-model"
branch_b = "dvc-complex-model"

# Ветки

In [11]:
print(branch_a, " vs ", branch_b)

dvc-base-model  vs  dvc-complex-model


Модели: 

* Логистическая регрессия с Tf-Idf (dvc-base-model).
* Случайный лес с truncated SVD поверх Tf-Idf (dvc-complex-model).

# Параметры

In [12]:
params_a = flatten_dict.flatten(dvc.api.params_show(rev=branch_a), reducer="dot")
params_b = flatten_dict.flatten(dvc.api.params_show(rev=branch_b), reducer="dot")

In [13]:
max_length = max(
    max(map(len, map(str, params_a.values()))), max(map(len, map(str, params_b.values())))
)

max_key_length = max(
    max(map(len, map(str, params_a.keys()))), max(map(len, map(str, params_b.keys())))
)

In [14]:
print(f"{'Name':<{max_key_length}} {branch_a:<{max_length}} {branch_b:<{max_length}}")

for key in sorted(set(params_a.keys()).union(params_b.keys())):
    is_a_missing = False
    is_b_missing = False

    if key not in params_a:
        is_a_missing = True

    val_a = params_a.get(key)

    if key not in params_b:
        is_b_missing = True

    val_b = params_b.get(key)

    if is_a_missing and is_b_missing or val_a == val_b:
        continue

    if is_a_missing:
        val_a = "-"
    if is_b_missing:
        val_b = "-"

    print(f"{key:<{max_key_length}} {val_a:<{max_length}} {val_b:<{max_length}}")

Name                               dvc-base-model                                                       dvc-complex-model                                                   
model.cls.estimator.C              100                                                                  -                                                                   
model.cls.estimator._target_       sklearn.linear_model.LogisticRegression                              sklearn.ensemble.RandomForestClassifier                             
model.cls.estimator.max_depth      -                                                                    6                                                                   
model.cls.estimator.n_estimators   -                                                                    100                                                                 
model.cls.estimator.n_jobs         -                                                                    -1                             

#  Метрики

Метрики на тесте и валидации получились хуже чем с базовой моделью.

In [15]:
metrics_a = dvc.api.metrics_show(rev=branch_a)
metrics_b = dvc.api.metrics_show(rev=branch_b)

In [16]:
print(f"{'Name':<20} {branch_a:<30} {branch_b:<30} {'Diff':<30}")

for metric_name in metrics_a:
    val_a = metrics_a[metric_name]
    val_b = metrics_b[metric_name]
    print(f"{metric_name:<20} {val_a:<30.4f} {val_b:<30.4f} {val_b - val_a:<30.4f}")

Name                 dvc-base-model                 dvc-complex-model              Diff                          
val_roc_auc          0.9941                         0.9723                         -0.0217                       
test_roc_auc         0.9897                         0.9745                         -0.0152                       
train_roc_auc        0.9896                         0.9980                         0.0084                        
